In [ ]:
import JSON
using Plots
using HDF5
using SparseArrays
using IterativeSolvers
using LinearAlgebra
using Statistics
include("../scripts/tools.jl")
include("../scripts/get_grid.jl")

In [ ]:
border = import_border("../data/border.json")
dx = 20
Nx, Ny, xrange, yrange, isgrid, isborder, n = get_grid(border, Float64(dx))

In [ ]:
# get indices of lattice nodes
id_xy = findall(isgrid)
id_x = Int64.(zeros(length(id_xy)))
id_y = Int64.(zeros(length(id_xy)))
for i in 1:length(id_xy)
    id_y[i] = id_xy[i][1]
    id_x[i] = id_xy[i][2]
end

Nbus = length(id_x)

In [ ]:
# get the list of edges and susceptance vector
epsilon = Int64.(zeros(Int64(1E6),2))
b = zeros(Int64(1E6))
k = 1

for i=1:length(id_x)
    if(isgrid[id_y[i], id_x[i]+1])
        id = findall((id_y .== id_y[i]) .& (id_x .== (id_x[i]+1)))
        epsilon[k,:] = [i id]
        b[k] = dx^2
        k += 1
    end
    if(isgrid[id_y[i]+1, id_x[i]])
        id = findall((id_y .== (id_y[i]+1)) .& (id_x .== id_x[i]))
        epsilon[k,:] = [i id]
        b[k] = dx^2
        k += 1
    end
end

N = k-1
epsilon = epsilon[1:N,:]
b = b[1:N]

B = sparse([epsilon[:,1];epsilon[:,2]],[1:N; 1:N],[-ones(N); ones(N)])
Db = sparse(1:N, 1:N, b)
L = B * Db * B'

In [ ]:
# If L is small enough for a direct eigendecomposition
minimum(diag(L))
@time begin
    eig = eigen(Array{Float64,2}(L))
    V_ref = eig.vectors
    D_ref = eig.values;
end

In [ ]:
# If not
# Iterative algorithm to obtain the Ntrunc lowest eigenvalues and -vectors

Ntrunc = 100
D = zeros(Ntrunc)
V = zeros(Nbus, Ntrunc)

@time begin
    for i=1:Ntrunc
        println(i)
        lambda_old = Inf
        if(i > 1)
            #lambda = D[i-1]
            lambda = 0.001
        else
            lambda = 0.001
        end
        x = rand(Nbus)
        for j = 1:i-1
            x = x - dot(x,V[:,j]) * V[:,j]
        end

        x = x .- mean(x)
        x = x ./ norm(x)

        iter = 1
        while((abs(lambda_old - lambda) / abs(lambda) > 1E-4) & (iter < 1E3))
            Lt = L - sparse(1:Nbus, 1:Nbus, lambda*ones(Nbus))
            x = minres(Lt, x)
            for j = 1:i-1
                x = x - dot(x, V[:,j]) * V[:,j]
            end
            x = x .- mean(x)
            x = x ./ norm(x)
            lambda_old = lambda
            lambda = dot(x, L * x)
            iter += 1
        end
        V[:,i] = x
        D[i] = lambda
    end
end
id_sort = sortperm(D)
D = D[id_sort]
V = V[:,id_sort];

In [ ]:
# plot a mode
a = 2
mode = zeros(Ny, Nx)
mode[id_xy] = V[:,a]
do_plot(isgrid, mode)

In [ ]:
# save the results
fid = h5open("../numerics/eigen_" * string(dx) * ".h5","w")
write(fid,"V", V_ref)
write(fid,"D", D_ref)
write(fid,"coord", [xrange[id_x] yrange[id_y]])
write(fid,"Nx", Nx)
write(fid,"Ny", Ny)
write(fid,"id_x", id_x)
write(fid,"id_y", id_y)
write(fid,"isgrid", Int64.(isgrid))
close(fid)